## 15-Puzzle



In [15]:
import itertools

### Function that checks if a permutation is even.


In [28]:
def is_permutation(a):
    '''
    Check if numbers in a are permutation - each number should 0 <= k < n and each number should be unique
    '''
    return set(a) == set(range(len(a)))

def is_even(p):
    '''
    Check if numbers contained in array can be sorted into 1,...n with even number of transpositions.
    '''
    transpositions = []
    for i in range(len(p)):
        if p[i] == i:
            continue
        # find where i is in (i+1 ... n-1) slots
        for j in range(i+1, len(p)):
            if p[j] == i:
                p[i], p[j] = p[j], p[i]
                transpositions.append((i, j))
    return len(transpositions) % 2 == 0

p = [1, 3, 0, 2]; print(p, is_even(p))
p = [0,3,2,4,5,6,7,1,9,8]; print(p, is_even(p))

[0, 1, 2, 3] False
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] True


### 15 Puzzle Representation
We have shown that a position in the 15-puzzle is unsolvable if the corresponding permutation of 15 objects is odd. We have not shown yet that the reverse statement is true, i.e., that for every even permutation the puzzle is solvable. It is indeed true, and the challenge now is to write a program that produces a sequence of moves for every solvable configuration.

Unfortunately, our representation of permutations in python starts with 0, and the numbering of pieces starts with 1. To make the output more readable, let us agree that the empty is represented by 0, and the other pieces are listed according to their labels in the "reading order", so the standard position is represented as [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] while the impossible configuration we discussed is represented as [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14, 0]. So the position is now represented by a permutation of 0..15, and we assume for simplicity that in the initial position the last number (bottom right cell) is 0.

Getting such a sequence as input that corresponds to a solvable configuration, your program should output a sequence of moves that transform this configuration into a standard one. Each move is represented by an integer, a number on the piece that is moved

For example, you may check that for the position [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0] one of the solution is [15, 14, 10, 13, 9, 10, 14, 15]



In [31]:
def solve(p, g):
    '''
    find list of transpositions needed to solve p
    '''
    transpositions = []
    for i in range(len(p)):
        if p[i] == g[i]:
            continue
        # find where g[i] is in (i+1 ... n-1) slots
        for j in range(i+1, len(p)):
            if p[j] == g[i]:
                p[i], p[j] = p[j], p[i]
                transpositions.append((i, j))
    return transpositions

p = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]
goal = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]

solve(p, goal)

[(8, 9), (9, 12)]

In [126]:
def ix_of_zero(p):
    return [i for i in range(len(p)) if p[i] == 0][0]

def row_col(ix):
    return (ix // 4, ix % 4)

def shade_weight(p):
    '''
    Location of 0 (blank) on the board.
    If corresponding row, col on the board is even - return 1 else 0
    Referred to as x in sum(less(i)) + x
    Board representation: S - Shaded, N Nonshaded
    S N S N
    N S N S
    S N S N
    N S N S
    '''
    row, col = row_col(ix_of_zero(p))
    return int((row + col) % 2 == 0)

def less(p, i):
    '''
    Number of positions that are less than i to the right
    '''
    return sum(1 for j in range(i, len(p)) if p[j] < p[i] and p[j] != 0)

def is_solvable(p):
    '''
    Check if sum(less(i)) + x is even 
    '''
    check = sum(less(p, i) for i in range(i)) + shade_weight(p)
    return check % 2 == 0

def possible_moves(p):
    '''
    Find current ix of zero. Generate UP, DOWN, LEFT, RIGHT. Return valid options
    '''
    r, c = row_col(ix_of_zero(p))
    return [(nr, nc) for (nr, nc) in ((r, c+1), (r, c-1), (r-1, c), (r+1, c)) if (0 <= nr < 4) and (0 <= nc < 4)]


def apply_move(p, m):
    '''
    Exchange  zero with value in (nr, nc)
    '''
    nr, nc = m[0], m[1]
    ix_of_z = ix_of_zero(p)
    ix_of_n = nr * 4 + nc
    p[ix_of_z], p[ix_of_n] =  p[ix_of_n], p[ix_of_z]
    return p[ix_of_z]

def cost(p):
    '''
    number of cells that are not in right position (excluding zero)
    '''
    return sum(p[i] != (i+1) for i in range(len(p)-1))

    
def cost_mhd(p):
    '''
    Manhattan distance to goal
    '''
    mhd = 0
    for i in range(len(p)-1):
        if p[i] != (i+1):
            ix_of_target = [i for i in range(len(p)) if p[i] == i+1][0]
            cr, cc = row_col(i)
            tr, tc = row_col(ix_of_target)
            mhd += abs(cr-tr) + abs(cc - tc)
    return mhd 


def solution(p):
    assert is_solvable(p)

    next_board = p[:]
    next_move = None
    past_zero_pos = None
    sol = []
    iter_count = 0
    max_iter_count = 1024*10
    
    while cost(next_board) > 0:    
        p_moves = possible_moves(next_board)
        #print('Possible Moves: ', p_moves)
        if past_zero_pos is not None:
            p_moves = [m for m in p_moves if m != past_zero_pos]
            #print('Filetered Possible Moves: ', p_moves)

        past_zero_pos = row_col(ix_of_zero(next_board))
        cost_data = []
        for move in p_moves:
            np = next_board[:]
            v = apply_move(np, move)
            c = cost(np)
            #print('Move : ', move, ' B:', np, ' Cost: ', c)
            cost_data.append((c, move, np, v))

        # sort of cost
        cost_data = sorted(cost_data)
        c, next_move, next_board, v = cost_data[0]
        #print('Selected Move: ', next_move)
        #print(next_board)        
        sol.append(v)
        iter_count += 1
        if iter_count > max_iter_count:
            print(next_board)
            raise Exception(f"No convergence after {iter_count} iterations")
    return sol
        
g = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
p = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 0, 15]
p1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 13, 14, 15]
p = [1, 2, 3, 4, 5, 6, 8, 9, 10, 0, 11, 13, 14, 15, 12, 7]

print('Solvable: ', is_solvable(p))
solution(p)        

Solvable:  True
[6, 10, 12, 4, 7, 0, 15, 14, 9, 8, 11, 5, 13, 3, 2, 1]


Exception: No convergence after 10241 iterations

In [81]:
p = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 0, 15]




4

In [95]:
[possible_moves(p) 

{(2, 2), (3, 1), (3, 3)}

In [36]:


p = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]
[(p[i], less(p, i)) for i in range(len(p))]

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (13, 4),
 (9, 0),
 (11, 1),
 (12, 1),
 (10, 0),
 (14, 0),
 (15, 0),
 (0, 0)]

https://www.youtube.com/watch?v=AM3oveV6-bo